## CLEAN UP HAPPINESS DATA

In [37]:
#IMPORT DEPENDECIES
import pandas as pd 
from sqlalchemy import create_engine
import csv
from datetime import date as dt
import pymysql

In [38]:
pymysql.install_as_MySQLdb()

In [39]:
Happiness_file = "Resources/Happiness2016.csv"
Happiness_df = pd.read_csv(Happiness_file)
Happiness_df.head()

,Country,Region,Happiness Rank,Happiness Score,Lower Confidence Interval,Upper Confidence Interval,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Denmark,Western Europe,1,7.526,7.460,7.592,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939
1,Switzerland,Western Europe,2,7.509,7.428,7.590,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463
2,Iceland,Western Europe,3,7.501,7.333,7.669,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137
3,Norway,Western Europe,4,7.498,7.421,7.575,1.57744,1.12690,0.79579,0.59609,0.35776,0.37895,2.66465
4,Finland,Western Europe,5,7.413,7.351,7.475,1.40598,1.13464,0.81091,0.57104,0.41004,0.25492,2.82596


In [40]:
Happiness_df2 = Happiness_df[['Country','Happiness Rank','Happiness Score','Economy (GDP per Capita)',]]
Happiness_df2.head()

,Country,Happiness Rank,Happiness Score,Economy (GDP per Capita)
0,Denmark,1,7.526,1.44178
1,Switzerland,2,7.509,1.52733
2,Iceland,3,7.501,1.42666
3,Norway,4,7.498,1.57744
4,Finland,5,7.413,1.40598


In [41]:
Final_Happiness_df =Happiness_df2.rename(columns= {
    "Country":"countries",
    "Happiness Rank": "happiness_rank", "Happiness Score": "happiness_score",
    "Economy (GDP per Capita)": "gdp"
})
Final_Happiness_df.head()

,countries,happiness_rank,happiness_score,gdp
0,Denmark,1,7.526,1.44178
1,Switzerland,2,7.509,1.52733
2,Iceland,3,7.501,1.42666
3,Norway,4,7.498,1.57744
4,Finland,5,7.413,1.40598


## CLEAN UP HUMAN FREEDOM INDEX DATA

In [42]:
freedom_data = pd.read_csv("Resources/hfi_cc_2018.csv")
freedom_data.head()

,year,ISO_code,countries,region,pf_rol_procedural,pf_rol_civil,pf_rol_criminal,pf_rol,pf_ss_homicide,pf_ss_disappearances_disap,...,ef_regulation_business_bribes,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,ef_score,ef_rank,hf_score,hf_rank,hf_quartile
0,2016,ALB,Albania,Eastern Europe,6.661503,4.547244,4.666508,5.291752,8.920429,10.0,...,4.050196,7.324582,7.074366,6.705863,6.906901,7.54,34.0,7.568140,48.0,2.0
1,2016,DZA,Algeria,Middle East & North Africa,NaN,NaN,NaN,3.819566,9.456254,10.0,...,3.765515,8.523503,7.029528,5.676956,5.268992,4.99,159.0,5.135886,155.0,4.0
2,2016,AGO,Angola,Sub-Saharan Africa,NaN,NaN,NaN,3.451814,8.060260,5.0,...,1.945540,8.096776,6.782923,4.930271,5.518500,5.17,155.0,5.640662,142.0,4.0
3,2016,ARG,Argentina,Latin America & the Caribbean,7.098483,5.791960,4.343930,5.744791,7.622974,10.0,...,3.260044,5.253411,6.508295,5.535831,5.369019,4.84,160.0,6.469848,107.0,3.0
4,2016,ARM,Armenia,Caucasus & Central Asia,NaN,NaN,NaN,5.003205,8.808750,10.0,...,4.575152,9.319612,6.491481,6.797530,7.378069,7.57,29.0,7.241402,57.0,2.0


In [43]:
freedom_data = freedom_data[(freedom_data['year'] == 2016)]
freedom_data = freedom_data[["countries","pf_score","pf_rank","ef_score","ef_rank","hf_score","hf_rank"]]
freedom_data.head()

,countries,pf_score,pf_rank,ef_score,ef_rank,hf_score,hf_rank
0,Albania,7.596281,57.0,7.54,34.0,7.568140,48.0
1,Algeria,5.281772,147.0,4.99,159.0,5.135886,155.0
2,Angola,6.111324,117.0,5.17,155.0,5.640662,142.0
3,Argentina,8.099696,42.0,4.84,160.0,6.469848,107.0
4,Armenia,6.912804,84.0,7.57,29.0,7.241402,57.0


## ESTABLISH SQL CONNECTION

In [44]:
connection = "root:root@127.0.0.1/project_db"
engine = create_engine(f'mysql://{connection}')

In [45]:
# Verify the table names
engine.table_names()

['freedom', 'happiness']

In [46]:
# Add the Pandas dataframe for Happiness into the our SQL table
Final_Happiness_df.to_sql(name='happiness', con=engine, if_exists='append', index=False)

In [47]:
# Add the Pandas dataframe for Human Freedom index into the our SQL table
freedom_data.to_sql(name='freedom', con=engine, if_exists='append', index=False)